In [ ]:
using CSV
using DataFrames
using DataStructures
using Glob
using JSON
using LargeScaleAnalysis
using Missings
using PyCall
using PyPlot
using Seaborn

In [ ]:
DateFormatter = pyimport("matplotlib.dates").DateFormatter
DayLocator = pyimport("matplotlib.dates").DayLocator
HourLocator = pyimport("matplotlib.dates").HourLocator;

In [ ]:
using Dates
function PyPlot.plot(df, model; ax = gca())
    cmap = Seaborn.seaborn.husl_palette(model["n_states"], l=0.7, s=0.9)
    ax.plot(unix2datetime.(df.timestamp), coalesce.(df.diff, NaN))
    for segment in model["segments"]
        span = unix2datetime.((segment["start_time"], segment["stop_time"]))
        ax.axvspan(span..., alpha = 0.3, color = cmap[parse(Int, segment["state"])])
    end
end

In [ ]:
files = glob("*.csv", "/blobs/datasets/imc_2019/manic/");

In [ ]:
file = "/blobs/datasets/imc_2019/manic/ory5-fr_194.149.163.90_149.11.115.13_174.csv"

In [ ]:
df = CSV.read(file, datarow = 2, header = ["timestamp", "diff"])
model = JSON.parsefile("$file.json")
monitor, near_ip, far_ip, asn = match(r".*/(.+)_(.+)_(.+)_(.+)\..+", file).captures

@pywith plt.style.context("classicthesis") begin
    fig, ax = subplots(figsize = (4.6, 0.4*4.6))
    plot(df, model)
    ax.set(
        title = "Free (AS 12322) \$\\rightarrow\$  Cogent (AS 174)\n$(monitor) / $(near_ip) / $(far_ip)",
        xlim = (DateTime(2018,4,2,6), DateTime(2018,4,6,6)),
        ylabel = "\$\\Delta\$ RTT (ms)"
    )
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    ax.xaxis.set_major_locator(HourLocator(byhour=[16]))
    fig.savefig("../figures/manic-free-cogent.pdf")
end

In [ ]:
groups = DefaultDict{String,Vector{Float64}}(() -> Float64[])
for segment in model["segments"]
    push!(groups[segment["state"]], df.diff[segment["start"]+1:segment["stop"]+1]...)
end

In [ ]:
@pywith plt.style.context("classicthesis") begin
    fig, ax = subplots(figsize = (4.6, 2.5))
    cmap = Seaborn.seaborn.husl_palette(model["n_states"], l=0.7, s=0.9)
    kdeplot(groups["1"], color = cmap[1], shade = true, label = "Congestion", ax = ax)
    kdeplot(groups["3"], color = cmap[3], shade = true, label = "Précongestion", ax = ax)
    ax.set(xlabel = "\$\\Delta\$ RTT (ms)", ylabel = "Densité")
    fig.savefig("../figures/manic-free-cogent-density.pdf")
end